# Data Analysis

This notebook is ***NEW CONTINUING*** from previous notebooks in the [notebooks](https://github.com/Data-Science-for-Linguists-2023/For-Reddit-Grammaticality-Analysis/tree/main/notebooks) folder. Therefore, Before reading through this notebook, I recommend going through [Data Collection](https://github.com/Data-Science-for-Linguists-2023/For-Reddit-Grammaticality-Analysis/blob/main/notebooks/dataCollection.ipynb) and then [Data Organization](https://github.com/Data-Science-for-Linguists-2023/For-Reddit-Grammaticality-Analysis/blob/main/notebooks/dataOrganization.ipynb)

**Outline**
1. [Setting Up](#Setting-Up)
2. [Dataframe Information](#Dataframe-Information)
3. [Language Tool Demonstration](#Language-Tool-Demonstration)
4. [Setting Up For Analysis](#Setting-Up-for-Analysis)
5. [Analysis](#Analysis)
6. [Analysis To Be Continued](#Analysis-To-Be-Continued)


In notebooks prior, I have worked on collecting the data from various subreddits and cleaning up that data in a dataframe. In this notebook, I will begin the process of data analysis. Specifically, I will be using [language-tool-python](https://pypi.org/project/language-tool-python/) to parse through the various posts in different subreddits. Let's begin!

## Setting Up

In [1]:
pip install language-tool-python

Note: you may need to restart the kernel to use updated packages.


In [2]:
# Start with importing our libraries
import language_tool_python as ltp  # Using this as a grammaticality parser
import pandas as pd 
import numpy as np
import nltk
import re

In [3]:
# This sets up our parsing tool
tool = ltp.LanguageTool('en-US')

Let's begin by reading in our final CSV files, created in the dataOrganization notebook.

In [4]:
# Using pandas to read csv files in 

legalAdvice = pd.read_csv("../final-data/finalLegalData.csv")
adulting = pd.read_csv('../final-data/finalAdData.csv')
medicine = pd.read_csv('../final-data/finalMedData.csv')
highschool = pd.read_csv('../final-data/finalHsData.csv')
broadway = pd.read_csv('../final-data/finalBwayData.csv')
pittsburgh = pd.read_csv('../final-data/finalPghData.csv')
rant = pd.read_csv('../final-data/finalRantData.csv')
ccq = pd.read_csv('../final-data/finalCcqData.csv')
anime = pd.read_csv('../final-data/finalAnimeData.csv')
eli5 = pd.read_csv('../final-data/finalElifData.csv')
college = pd.read_csv('../final-data/finalCollegeData.csv')
sports = pd.read_csv('../final-data/finalSportsData.csv')
crypto = pd.read_csv('../final-data/finalCryptoData.csv')
lawyertalk = pd.read_csv('../final-data/finalLawyerData.csv')
gaming = pd.read_csv('../final-data/finalGamingData.csv')

Removing the random column from the CSV files

In [5]:
# Drop the "Unnamed: 0" column
legalAdvice = legalAdvice.drop('Unnamed: 0', axis = 1)
adulting = adulting.drop('Unnamed: 0', axis = 1)
medicine = medicine.drop('Unnamed: 0', axis = 1)
highschool = highschool.drop('Unnamed: 0', axis = 1)
broadway = broadway.drop('Unnamed: 0', axis = 1)
pittsburgh = pittsburgh.drop('Unnamed: 0', axis = 1)
rant = rant.drop('Unnamed: 0', axis = 1)
ccq = ccq.drop('Unnamed: 0', axis = 1)
anime = anime.drop('Unnamed: 0', axis = 1)
eli5 = eli5.drop('Unnamed: 0', axis = 1)
college = college.drop('Unnamed: 0', axis = 1)
sports = sports.drop('Unnamed: 0', axis = 1)
crypto = crypto.drop('Unnamed: 0', axis = 1)
lawyertalk = lawyertalk.drop('Unnamed: 0', axis = 1)
gaming = gaming.drop('Unnamed: 0', axis = 1)

## Dataframe Information

Let's get a refresher on what the dataframe looks like. Each subreddit dataframe contains:
1. Title - The title of the post
2. Id - The ID of the post
3. Text - The actual contents of the post
4. Author - The user that wrote the post
5. Number of Comments - The number of comments the post has
6. Number of upvotes - The number of upvotes the post has
7. Ratio of Upvotes - The ratio of upvotes to downvotes

**1, 2 4**: Not super relevant for our purposes right now, mostly to keep track of these posts if they need to be searched on Reddit

**3**: Relevant for our purposes right now, will be working with this part shortly

**5 - 7**: Not super relevant to our purposes right now, will potentially work with this information in the future.

In [6]:
# Show some entries of the ccq(CS Career Questions) dataframe
ccq

,Title,Id,Text,Author,Number of Comments,Number of upvotes,Ratio of Upvotes
0,"Big N Discussion - March 19, 2023",11ve46y,Please use this thread to have discussions abo...,CSCQMods,7,5,0.73
1,"Daily Chat Thread - March 19, 2023",11ve5o1,"Please use this thread to chat, have casual di...",CSCQMods,0,1,0.60
2,Is it acceptable to do lunch 12-1pm at work? A...,11voie0,Asking as a new grad who is trying to understa...,TheCockatoo,214,225,0.74
3,Number of Open Tech Jobs has increased for 2 c...,11vqmgd,https://www.trueup.io/job-trend\n\nThis is a f...,TheCopyPasteLife,46,95,0.89
4,How to enforce good practices in my workplace?,11viy3c,"My team doesn't enforce good practices, and my...",Old-Fennel9061,58,149,0.91
...,...,...,...,...,...,...,...
1495,Nerves about starting first SWE Role,11k8to9,I’m graduating from a top CS university this s...,BringMeTheBRBS,3,0,0.33
1496,Reaching out to someone for help with a position,11k87kz,"Hello all, to cut to the chase, I was recently...",businessbee89,1,1,1.00
1497,Portfolio projects - better to create somethin...,11k7q3y,So I'm getting started creating a project for ...,GroundFallsOnly,4,5,1.00
1498,"SWEs in the UK, what is your day-to-day actual...",11k7dz7,We see lots of YouTube videos where young SWEs...,nonbog,7,3,0.80


Now, let's just focus in on one post from that subreddit

Posts can potentially contain a lot of characters that the grammaticality library may not like. By not removing them, this could lead to unfair results as *technically* these posts are grammatical, they just contain words such as "Reddit" or "Subreddit". Potentially, as I am working on this analysis, because there are so many cases to consider, I can look at later whether these types of errors fall within the same category so that I could just ignore that category.

## Language Tool Demonstration

In [7]:
onePost = ccq['Text'][3]
onePost

"https://www.trueup.io/job-trend\n\nThis is a follow up from [last week's post.](https://old.reddit.com/r/cscareerquestions/comments/11odfe7/number_of_open_tech_jobs_has_increased_for_the/) It definitely seems like the market is starting to turn around. I also have anecdotal evidence of my own. Feel free to add yours.\n\nPossible risks include reduced lending to startups due to regional bank liquidity. Also another wave of layoffs, like Facebook, but I think that Facebook's layoffs come from a dying business, not an industry-wide concern."

Essentially, for each error, it will put the entire error in parentheses, preceded by "Match". The tool says what type of error it is, provides a bit of an explanation, and then suggests fixes. For the post above, there are two errors.

In [8]:
# Use the tool to see what it will give us for the post above
oneCcqPost = tool.check(onePost)
oneCcqPost

[Match({'ruleId': 'VERB_NOUN_CONFUSION', 'message': 'When ‘follow-up’ is used as a noun or modifier, it needs to be hyphenated.', 'replacements': ['follow-up'], 'offsetInContext': 43, 'context': "...ps://www.trueup.io/job-trend  This is a follow up from [last week's post.](https://old.re...", 'offset': 43, 'errorLength': 9, 'category': 'COMPOUNDING', 'ruleIssueType': 'uncategorized', 'sentence': "This is a follow up from [last week's post.](https://old.reddit.com/r/cscareerquestions/comments/11odfe7/number_of_open_tech_jobs_has_increased_for_the/) It definitely seems like the market is starting to turn around."}),
 Match({'ruleId': 'SENT_START_CONJUNCTIVE_LINKING_ADVERB_COMMA', 'message': 'A comma may be missing after the conjunctive/linking adverb ‘Also’.', 'replacements': ['Also,'], 'offsetInContext': 43, 'context': '...tartups due to regional bank liquidity. Also another wave of layoffs, like Facebook,...', 'offset': 401, 'errorLength': 4, 'category': 'PUNCTUATION', 'ruleIssueType':

In [9]:
# This syntax will be important... will allow us to directly access the errors of each paragraph
oneCcqPost[0].ruleId

'VERB_NOUN_CONFUSION'

In [10]:
oneCcqPost[1].ruleId

'SENT_START_CONJUNCTIVE_LINKING_ADVERB_COMMA'

In [11]:
onePost = ccq['Text'][1]
onePost

"Please use this thread to chat, have casual discussions, and ask casual questions. Moderation will be light, but don't be a jerk.\n\nThis thread is posted **every day at midnight PST**. Previous Daily Chat Threads can be found [here](https://www.reddit.com/r/cscareerquestions/search?q=Daily+Chat+Thread&restrict_sr=on&sort=new&t=all)."

If the post has no errors, it will run silently like below!

In [12]:
oneCcqPost = tool.check(onePost)
oneCcqPost

[]

In [13]:
onePost = ccq['Text'][22]
onePost

"Crossposting from r/AskAcademia \\- thought I could get some useful knowledge from a different group of folks.\n\nI'm in a really tough situation and have been feeling acutely ill over making a decision. I would be really grateful for some advice.\n\nI graduated from a prestigious university with a bachelors in CS last spring, and have been working and making a really comfortable salary in a big tech company (return offer on a summer 2021 internship). I had worked with an assistant professor on ML research for the last two years and had a good and productive time; the professor really believed in me, and towards the last semester, suggested that I apply for PhD programs in that area.\n\nI did not believe that I could get into any PhD programs this cycle, as this particular subfield is extremely competitive, and so figured that I would not seriously decide whether or not to do a PhD unless I get in. I only targeted very competitive places and applied to my undergrad's fully-funded, res

As seen below however, words such as 'r/AskAcademia' are marked as a spelling mistake. This is problematic, as a lot of posts refer to r/subredditname, but that does not make them ungrammatical in the perspective that I wish to look at it. This poses a potential issue with the parser.

In [14]:
oneCcqPost = tool.check(onePost)
oneCcqPost

[Match({'ruleId': 'MORFOLOGIK_RULE_EN_US', 'message': 'Possible spelling mistake found.', 'replacements': ['Cross posting'], 'offsetInContext': 0, 'context': 'Crossposting from r/AskAcademia \\- thought I could g...', 'offset': 0, 'errorLength': 12, 'category': 'TYPOS', 'ruleIssueType': 'misspelling', 'sentence': 'Crossposting from r/AskAcademia \\- thought I could get some useful knowledge from a different group of folks.'}),
 Match({'ruleId': 'MORFOLOGIK_RULE_EN_US', 'message': 'Possible spelling mistake found.', 'replacements': ['Academia'], 'offsetInContext': 20, 'context': 'Crossposting from r/AskAcademia \\- thought I could get some useful know...', 'offset': 20, 'errorLength': 11, 'category': 'TYPOS', 'ruleIssueType': 'misspelling', 'sentence': 'Crossposting from r/AskAcademia \\- thought I could get some useful knowledge from a different group of folks.'}),
 Match({'ruleId': 'ENGLISH_WORD_REPEAT_BEGINNING_RULE', 'message': 'Three successive sentences begin with the same word. C

However, if we use regex to substitute out "reddit" in strings, the parser does not mark this as an error. This may be helpful in my pursuit of finding meaninful grammatical errors in the future.

In [15]:
test = "This is a Reddit post."
test = re.sub(r"[rR]\/\S+", "--", test)
test

'This is a Reddit post.'

In [16]:
oneTestPost = tool.check(test)
oneTestPost

[]

## Setting Up for Analysis

Now, for analysis purposes, we want to grab all of the CSV files "Text" columns and put them into a list for iteration

In [17]:
# Creating list of 'Text' values

allSportsVals = list(sports['Text'].values)
allLegalVals = list(legalAdvice['Text'].values)
allAdultVals = list(adulting['Text'].values)
allMedVals = list(medicine['Text'].values)
allHsVals = list(highschool['Text'].values)
allBwayVals = list(broadway['Text'].values)
allPghVals = list(pittsburgh['Text'].values)
allRantVals = list(rant['Text'].values)
allCcqVals = list(ccq['Text'].values)
allAnimeVals = list(anime['Text'].values)
allEli5Vals = list(eli5['Text'].values)
allCollegeVals = list(college['Text'].values)
allCryptoVals = list(crypto['Text'].values)
allLawyerVals = list(lawyertalk['Text'].values)
allGamingVals = list(gaming['Text'].values)

Now, we want to append the output of the language tool on each item in the list containing text from each subreddit

In [20]:
# Empty lists
sportsErrors = []
legalErrors = []
adultErrors = []
medErrors = []
hsErrors = []
bwayErrors = []
pghErrors = []
rantErrors = []
ccqErrors = []
animeErrors = []
eli5Errors = []
collegeErrors = []
cryptoErrors = []
lawyerErrors = [] 
gamingErrors = []



In [22]:
# Appending the output of the language tool to a list
[sportsErrors.append(tool.check(x)) for x in allSportsVals]
print('Done processing')

Done processing


In [23]:
[legalErrors.append(tool.check(x)) for x in allLegalVals]
print('Done processing')

Done processing


In [24]:
[adultErrors.append(tool.check(x)) for x in allAdultVals]
print('Done processing')

Done processing


In [25]:
[medErrors.append(tool.check(x)) for x in allMedVals]
print('Done processing')

Done processing


In [26]:
[hsErrors.append(tool.check(x)) for x in allHsVals]
print('Done processing')

Done processing


In [27]:
[bwayErrors.append(tool.check(x)) for x in allBwayVals]
print('Done processing')

Done processing


In [28]:
[pghErrors.append(tool.check(x)) for x in allPghVals]
print('Done processing')

Done processing


In [29]:
[rantErrors.append(tool.check(x)) for x in allRantVals]
print('Done processing')

Done processing


In [30]:
[ccqErrors.append(tool.check(x)) for x in allCcqVals]
print('Done processing')

Done processing


In [31]:
[animeErrors.append(tool.check(x)) for x in allAnimeVals]
print('Done processing')

Done processing


In [32]:
[eli5Errors.append(tool.check(x)) for x in allEli5Vals]
print('Done processing')

Done processing


In [33]:
[collegeErrors.append(tool.check(x)) for x in allCollegeVals]
print('Done processing')

Done processing


In [34]:
[cryptoErrors.append(tool.check(x)) for x in allCryptoVals]
print('Done processing')

Done processing


In [35]:
[lawyerErrors.append(tool.check(x)) for x in allLawyerVals]
print('Done processing')

Done processing


In [36]:
[gamingErrors.append(tool.check(x)) for x in allGamingVals]
print('Done processing')

Done processing


In [37]:
# Just making sure it is formatted correctly

collegeErrors[0]

[Match({'ruleId': 'MD_BASEFORM', 'message': 'The modal verb ‘will’ requires the verb’s base form.', 'replacements': ['become'], 'offsetInContext': 43, 'context': '...-2023 school year: 2022-2023 FAFSA will became available October 1, 2021. Requires 202...', 'offset': 522, 'errorLength': 6, 'category': 'GRAMMAR', 'ruleIssueType': 'grammar', 'sentence': '2022-2023 school year: 2022-2023 FAFSA will became available October 1, 2021.'}),
 Match({'ruleId': 'ENGLISH_WORD_REPEAT_BEGINNING_RULE', 'message': 'Three successive sentences begin with the same word. Consider rewording the sentence or use a thesaurus to find a synonym.', 'replacements': [], 'offsetInContext': 43, 'context': '...ir own FSA account, they must use that. If your parent does not have an SSN, they ...', 'offset': 997, 'errorLength': 2, 'category': 'STYLE', 'ruleIssueType': 'style', 'sentence': 'If your parent does not have an SSN, they must print and sign the signature page manually, then mail it in.'}),
 Match({'ruleId': '

For now, I want to focus in on the RuleId(ex: MORFOLOGIK_RULE_EN_US). Let's grab those from the list

In [38]:
# For each subreddit, append the rule id to the list
simplifiedColErrors = []
[simplifiedColErrors.append(y.ruleId) for x in collegeErrors for y in x]

simplifiedSportsErrors = []
[simplifiedSportsErrors.append(y.ruleId) for x in sportsErrors for y in x]

simplifiedLegalErrors = []
[simplifiedLegalErrors.append(y.ruleId) for x in legalErrors for y in x]

simplifiedAdultErrors = []
[simplifiedAdultErrors.append(y.ruleId) for x in adultErrors for y in x]

simplifiedMedErrors = []
[simplifiedMedErrors.append(y.ruleId) for x in medErrors for y in x]

simplifiedHsErrors = []
[simplifiedHsErrors.append(y.ruleId) for x in hsErrors for y in x]

simplifiedBwayErrors = []
[simplifiedBwayErrors.append(y.ruleId) for x in bwayErrors for y in x]

simplifiedPghErrors = []
[simplifiedPghErrors.append(y.ruleId) for x in pghErrors for y in x]

simplifiedRantErrors = []
[simplifiedRantErrors.append(y.ruleId) for x in rantErrors for y in x]

simplifiedCcqErrors = []
[simplifiedCcqErrors.append(y.ruleId) for x in ccqErrors for y in x]

simplifiedAnimeErrors = []
[simplifiedAnimeErrors.append(y.ruleId) for x in animeErrors for y in x]

simplifiedEli5Errors = []
[simplifiedEli5Errors.append(y.ruleId) for x in eli5Errors for y in x]

simplifiedCryptoErrors = []
[simplifiedCryptoErrors.append(y.ruleId) for x in cryptoErrors for y in x]

simplifiedLawyerErrors = []
[simplifiedLawyerErrors.append(y.ruleId) for x in lawyerErrors for y in x]

simplifiedGamingErrors = []
[simplifiedGamingErrors.append(y.ruleId) for x in gamingErrors for y in x]

print("Done")

Done


In [39]:
simplifiedGamingErrors[:10]

['WHOSE_DT',
 'MORFOLOGIK_RULE_EN_US',
 'MORFOLOGIK_RULE_EN_US',
 'MORFOLOGIK_RULE_EN_US',
 'LIFE_COMPOUNDS',
 'OVER_COMPOUNDS',
 'ENGLISH_WORD_REPEAT_BEGINNING_RULE',
 'WHITESPACE_RULE',
 'MISSING_HYPHEN',
 'WHITESPACE_RULE']

## Analysis

Ok that's it! Thats the entire project! Anime clearly is the most ungrammatical subreddit out of the 15. Just kidding...time to look further into all of this!

In [40]:
# Printing the length of each list, which is the number of errors in each subreddit

print(len(simplifiedColErrors))
print(len(simplifiedSportsErrors)) 
print(len(simplifiedLegalErrors)) 
print(len(simplifiedAdultErrors ))
print(len(simplifiedMedErrors))
print(len(simplifiedHsErrors))
print(len(simplifiedBwayErrors ))
print(len(simplifiedPghErrors ))
print(len(simplifiedRantErrors ))
print(len(simplifiedCcqErrors ))
print(len(simplifiedAnimeErrors))
print(len(simplifiedEli5Errors ))
print(len(simplifiedCryptoErrors))
print(len(simplifiedLawyerErrors))
print(len(simplifiedGamingErrors ))

6535
24724
7210
8008
4957
8321
5391
3040
10431
5884
14851
1791
13602
3943
6238


Listed below is all of the possible errors across all of the subreddits. There are a lot of errors, and some of them aren't super clear as to what they are talking about.

In [41]:
allErrors = set(simplifiedColErrors + simplifiedSportsErrors + simplifiedLegalErrors + simplifiedAdultErrors 
    + simplifiedMedErrors + simplifiedHsErrors + simplifiedBwayErrors + simplifiedPghErrors + simplifiedRantErrors
   + simplifiedCcqErrors + simplifiedAnimeErrors + simplifiedEli5Errors + simplifiedCryptoErrors + 
   simplifiedLawyerErrors + simplifiedGamingErrors)
print(allErrors)

{'PER_SE', 'MUST_HAVE_TO', 'TAKEAWAY', 'THIS_MISSING_VERB', 'IF_OF', 'WE_LL_WELL', 'STAIRS_COMPOUNDS', 'AFFECT_EFFECT', 'MOST_SOME_OF_NNS', 'THE_BEST_WAY', 'THIRD_WORLD', 'PRP_JJ', 'MASS_AGREEMENT', 'WOULD_BE_JJ_VB', 'FR', 'GO_GERUND', 'TO_TOO', 'DAYTIME', 'LEARN_NNNNS_ON_DO', 'BACHELOR_ABBR', 'SENT_START_ARE_NOT_ARENT_FORMAL', 'ON_COMPOUNDS', 'HAND_FULL_COMPOUND', 'HIGH_END_HYPHEN', 'ALL_OF_THE_SUDDEN', 'HAND_AND_HAND', 'EN_WORD_COHERENCY', 'COMMA_TAG_QUESTION', 'GATHER_UP', 'NOTE_TAKING_HYPHEN', 'LIFE_LIVE', 'BEEN_PART_AGREEMENT', 'CC_NNP_VBP', 'FEWER_LESS', 'WIFI', 'CASTED', 'TH_THORIUM', 'MOST_SUPERLATIVE', 'ROYAL_MAIL', 'PRP_THE', 'IN_OR_WITH_REGARDS_TO_OF', 'LESS_MORE_THEN', 'COMMA_COMPOUND_SENTENCE_3', 'THE_SUPERLATIVE', 'TWO_CONNECTED_MODAL_VERBS', 'DUN_DONT', 'A_COLLECTIVE_OF_NN', 'NOT_SURE_IT_WORKS', 'ACHE_COMPOUNDS', 'RETURN_BACK', 'ALA_MODE', 'WHOS', 'HIPAA', 'PRP_VB', 'FOR_VB', 'DATE_WEEKDAY_WITHOUT_YEAR', 'WORLDS_BEST', 'TO_DO_HYPHEN', 'IT_ITS', 'FOR_GODS_SAKE_COMMA', 'WE

This part is super messy, but I needed a way to get both the error and the count of that error in each subreddit into a list for each subreddit

In [42]:
# Declaring a bunch of lists
col, sp, le, ad, med, hs, bway, pgh, rant, ccq, ani, eli5, cry, law, gam = [], [], [], [], [], [], [], [], [], [], [], [], [], [], []

# For each error, coint how many times that error occurs and append the error and the count to the list
for x in allErrors:
    
    y = simplifiedColErrors.count(x)
    col.append((x, y))
    
    y = simplifiedSportsErrors.count(x)
    sp.append((x, y))
    
    y = simplifiedLegalErrors.count(x)
    le.append((x, y))
    
    y = simplifiedAdultErrors.count(x)
    ad.append((x, y))
    
    y = simplifiedMedErrors.count(x)
    med.append((x, y))
    
    y = simplifiedHsErrors.count(x)
    hs.append((x, y))
    
    y = simplifiedBwayErrors.count(x)
    bway.append((x, y))
    
    y = simplifiedPghErrors.count(x)
    pgh.append((x, y))
    
    y = simplifiedRantErrors.count(x)
    rant.append((x, y))
    
    y = simplifiedCcqErrors.count(x)
    ccq.append((x, y))
    
    y = simplifiedAnimeErrors.count(x)
    ani.append((x, y))
    
    y = simplifiedEli5Errors.count(x)
    eli5.append((x, y))
    
    y = simplifiedCryptoErrors.count(x)
    cry.append((x, y))
    
    y = simplifiedLawyerErrors.count(x)
    law.append((x, y))
    
    y = simplifiedGamingErrors.count(x)
    gam.append((x, y))

In [43]:
col.sort(key = lambda x: x[1])
col[1011:]

[('GONNA', 45),
 ('MISSING_HYPHEN', 49),
 ('SENT_START_CONJUNCTIVE_LINKING_ADVERB_COMMA', 55),
 ('IDK', 61),
 ('WANNA', 63),
 ('COMMA_PARENTHESIS_WHITESPACE', 90),
 ('EN_COMPOUNDS', 112),
 ('COMMA_COMPOUND_SENTENCE_2', 126),
 ('WHITESPACE_RULE', 144),
 ('EN_CONTRACTION_SPELLING', 171),
 ('UPPERCASE_SENTENCE_START', 439),
 ('ENGLISH_WORD_REPEAT_BEGINNING_RULE', 507),
 ('I_LOWERCASE', 961),
 ('MORFOLOGIK_RULE_EN_US', 1109),
 ('COMMA_COMPOUND_SENTENCE', 1299)]

In [44]:
sp.sort(key = lambda x: x[1])
sp[1011:]

[('ENGLISH_WORD_REPEAT_BEGINNING_RULE', 66),
 ('POSSESSIVE_APOSTROPHE', 92),
 ('ENGLISH_WORD_REPEAT_RULE', 94),
 ('CD_NN', 98),
 ('EN_COMPOUNDS', 102),
 ('COMMA_PARENTHESIS_WHITESPACE', 106),
 ('EN_CONTRACTION_SPELLING', 134),
 ('SENTENCE_WHITESPACE', 140),
 ('UPPERCASE_SENTENCE_START', 142),
 ('COMMA_COMPOUND_SENTENCE_2', 174),
 ('EN_DIACRITICS_REPLACE', 188),
 ('EN_UNPAIRED_BRACKETS', 200),
 ('COMMA_COMPOUND_SENTENCE', 584),
 ('WHITESPACE_RULE', 2426),
 ('MORFOLOGIK_RULE_EN_US', 18742)]

In [45]:
le.sort(key = lambda x: x[1])
le[1011:]

[('EN_UNPAIRED_BRACKETS', 51),
 ('ENGLISH_WORD_REPEAT_RULE', 53),
 ('SENT_START_CONJUNCTIVE_LINKING_ADVERB_COMMA', 65),
 ('MISSING_HYPHEN', 80),
 ('POSSESSIVE_APOSTROPHE', 83),
 ('COMMA_PARENTHESIS_WHITESPACE', 118),
 ('EN_CONTRACTION_SPELLING', 143),
 ('EN_COMPOUNDS', 197),
 ('COMMA_COMPOUND_SENTENCE_2', 200),
 ('WHITESPACE_RULE', 282),
 ('ENGLISH_WORD_REPEAT_BEGINNING_RULE', 372),
 ('UPPERCASE_SENTENCE_START', 391),
 ('I_LOWERCASE', 590),
 ('MORFOLOGIK_RULE_EN_US', 1112),
 ('COMMA_COMPOUND_SENTENCE', 1776)]

In [46]:
ad.sort(key = lambda x: x[1])
ad[1011:]

[('ENGLISH_WORD_REPEAT_RULE', 49),
 ('YEAR_OLD_HYPHEN', 57),
 ('ETC_PERIOD', 62),
 ('IDK', 64),
 ('DOUBLE_PUNCTUATION', 66),
 ('COMMA_PARENTHESIS_WHITESPACE', 88),
 ('EN_CONTRACTION_SPELLING', 162),
 ('COMMA_COMPOUND_SENTENCE_2', 165),
 ('EN_COMPOUNDS', 169),
 ('UPPERCASE_SENTENCE_START', 361),
 ('WHITESPACE_RULE', 608),
 ('I_LOWERCASE', 714),
 ('ENGLISH_WORD_REPEAT_BEGINNING_RULE', 720),
 ('MORFOLOGIK_RULE_EN_US', 1458),
 ('COMMA_COMPOUND_SENTENCE', 1616)]

In [47]:
med.sort(key = lambda x: x[1])
med[1011:]

[('HE_VERB_AGR', 29),
 ('EN_SPLIT_WORDS_HYPHEN', 29),
 ('SENT_START_CONJUNCTIVE_LINKING_ADVERB_COMMA', 35),
 ('I_LOWERCASE', 44),
 ('ETC_PERIOD', 44),
 ('COMMA_PARENTHESIS_WHITESPACE', 48),
 ('EN_UNPAIRED_BRACKETS', 53),
 ('COMMA_COMPOUND_SENTENCE_2', 76),
 ('EN_CONTRACTION_SPELLING', 81),
 ('UPPERCASE_SENTENCE_START', 104),
 ('EN_COMPOUNDS', 115),
 ('ENGLISH_WORD_REPEAT_BEGINNING_RULE', 180),
 ('WHITESPACE_RULE', 397),
 ('COMMA_COMPOUND_SENTENCE', 483),
 ('MORFOLOGIK_RULE_EN_US', 2399)]

In [48]:
hs.sort(key = lambda x: x[1])
hs[1011:]

[('IT_IS', 44),
 ('SENT_START_CONJUNCTIVE_LINKING_ADVERB_COMMA', 54),
 ('EN_COMPOUNDS', 56),
 ('WANNA', 81),
 ('IDK', 85),
 ('GONNA', 89),
 ('COMMA_PARENTHESIS_WHITESPACE', 104),
 ('COMMA_COMPOUND_SENTENCE_2', 127),
 ('WHITESPACE_RULE', 205),
 ('EN_CONTRACTION_SPELLING', 264),
 ('ENGLISH_WORD_REPEAT_BEGINNING_RULE', 448),
 ('UPPERCASE_SENTENCE_START', 783),
 ('COMMA_COMPOUND_SENTENCE', 1220),
 ('I_LOWERCASE', 1371),
 ('MORFOLOGIK_RULE_EN_US', 2089)]

In [49]:
bway.sort(key = lambda x: x[1])
bway[1011:]

[('EN_UNPAIRED_BRACKETS', 35),
 ('HE_VERB_AGR', 39),
 ('ENGLISH_WORD_REPEAT_RULE', 39),
 ('EN_CONTRACTION_SPELLING', 46),
 ('SENT_START_CONJUNCTIVE_LINKING_ADVERB_COMMA', 51),
 ('EN_COMPOUNDS', 57),
 ('COMMA_PARENTHESIS_WHITESPACE', 71),
 ('EN_DIACRITICS_REPLACE', 100),
 ('COMMA_COMPOUND_SENTENCE_2', 117),
 ('I_LOWERCASE', 144),
 ('ENGLISH_WORD_REPEAT_BEGINNING_RULE', 149),
 ('WHITESPACE_RULE', 159),
 ('UPPERCASE_SENTENCE_START', 232),
 ('COMMA_COMPOUND_SENTENCE', 776),
 ('MORFOLOGIK_RULE_EN_US', 2428)]

In [50]:
pgh.sort(key = lambda x: x[1])
pgh[1011:]

[('MISSING_HYPHEN', 23),
 ('YEAR_OLD_HYPHEN', 24),
 ('SENT_START_CONJUNCTIVE_LINKING_ADVERB_COMMA', 33),
 ('EN_CONTRACTION_SPELLING', 34),
 ('DOUBLE_PUNCTUATION', 36),
 ('COMMA_PARENTHESIS_WHITESPACE', 45),
 ('EN_COMPOUNDS', 56),
 ('I_LOWERCASE', 71),
 ('UPPERCASE_SENTENCE_START', 72),
 ('COMMA_COMPOUND_SENTENCE_2', 72),
 ('HE_VERB_AGR', 100),
 ('ENGLISH_WORD_REPEAT_BEGINNING_RULE', 119),
 ('WHITESPACE_RULE', 141),
 ('COMMA_COMPOUND_SENTENCE', 420),
 ('MORFOLOGIK_RULE_EN_US', 1166)]

In [51]:
rant.sort(key = lambda x: x[1])
rant[1011:]

[('CANT', 64),
 ('IDK', 67),
 ('IT_IS', 76),
 ('DOUBLE_PUNCTUATION', 89),
 ('GONNA', 106),
 ('COMMA_PARENTHESIS_WHITESPACE', 168),
 ('COMMA_COMPOUND_SENTENCE_2', 176),
 ('WHITESPACE_RULE', 247),
 ('EN_COMPOUNDS', 274),
 ('EN_CONTRACTION_SPELLING', 371),
 ('ENGLISH_WORD_REPEAT_BEGINNING_RULE', 605),
 ('UPPERCASE_SENTENCE_START', 761),
 ('I_LOWERCASE', 1237),
 ('COMMA_COMPOUND_SENTENCE', 1661),
 ('MORFOLOGIK_RULE_EN_US', 2300)]

In [52]:
ccq.sort(key = lambda x: x[1])
ccq[1011:]

[('AM_I', 35),
 ('GONNA', 38),
 ('MISSING_HYPHEN', 49),
 ('EN_A_VS_AN', 64),
 ('SENT_START_CONJUNCTIVE_LINKING_ADVERB_COMMA', 78),
 ('COMMA_PARENTHESIS_WHITESPACE', 86),
 ('COMMA_COMPOUND_SENTENCE_2', 123),
 ('UPPERCASE_SENTENCE_START', 126),
 ('EN_COMPOUNDS', 127),
 ('EN_CONTRACTION_SPELLING', 140),
 ('I_LOWERCASE', 303),
 ('WHITESPACE_RULE', 433),
 ('ENGLISH_WORD_REPEAT_BEGINNING_RULE', 468),
 ('COMMA_COMPOUND_SENTENCE', 1039),
 ('MORFOLOGIK_RULE_EN_US', 1584)]

In [53]:
ani.sort(key = lambda x: x[1])
ani[1011:]

[('SENT_START_CONJUNCTIVE_LINKING_ADVERB_COMMA', 61),
 ('ENGLISH_WORD_REPEAT_RULE', 64),
 ('ARROWS', 68),
 ('COMMA_COMPOUND_SENTENCE_2', 79),
 ('EN_UNPAIRED_BRACKETS', 82),
 ('ENGLISH_WORD_REPEAT_BEGINNING_RULE', 87),
 ('EN_COMPOUNDS', 89),
 ('EN_CONTRACTION_SPELLING', 105),
 ('COMMA_PARENTHESIS_WHITESPACE', 178),
 ('UPPERCASE_SENTENCE_START', 230),
 ('SENTENCE_WHITESPACE', 234),
 ('I_LOWERCASE', 319),
 ('WHITESPACE_RULE', 422),
 ('COMMA_COMPOUND_SENTENCE', 444),
 ('MORFOLOGIK_RULE_EN_US', 11050)]

In [54]:
eli5.sort(key = lambda x: x[1])
eli5[1011:]

[('IDK', 15),
 ('LETS_LET', 16),
 ('EN_UNPAIRED_BRACKETS', 18),
 ('HE_VERB_AGR', 21),
 ('SENT_START_CONJUNCTIVE_LINKING_ADVERB_COMMA', 21),
 ('COMMA_PARENTHESIS_WHITESPACE', 23),
 ('UNIT_SPACE', 24),
 ('EN_COMPOUNDS', 29),
 ('EN_CONTRACTION_SPELLING', 48),
 ('WHITESPACE_RULE', 59),
 ('UPPERCASE_SENTENCE_START', 64),
 ('I_LOWERCASE', 78),
 ('COMMA_COMPOUND_SENTENCE_2', 89),
 ('COMMA_COMPOUND_SENTENCE', 219),
 ('MORFOLOGIK_RULE_EN_US', 632)]

In [55]:
cry.sort(key = lambda x: x[1])
cry[1011:]

[('SENTENCE_WHITESPACE', 69),
 ('ENGLISH_WORD_REPEAT_RULE', 71),
 ('DOUBLE_PUNCTUATION', 78),
 ('SENT_START_CONJUNCTIVE_LINKING_ADVERB_COMMA', 107),
 ('EN_CONTRACTION_SPELLING', 138),
 ('EN_UNPAIRED_BRACKETS', 148),
 ('COMMA_COMPOUND_SENTENCE_2', 153),
 ('ENGLISH_WORD_REPEAT_BEGINNING_RULE', 172),
 ('UPPERCASE_SENTENCE_START', 216),
 ('EN_COMPOUNDS', 216),
 ('I_LOWERCASE', 243),
 ('COMMA_PARENTHESIS_WHITESPACE', 273),
 ('COMMA_COMPOUND_SENTENCE', 858),
 ('WHITESPACE_RULE', 2555),
 ('MORFOLOGIK_RULE_EN_US', 6229)]

In [56]:
law.sort(key = lambda x: x[1])
law[1011:]

[('POSSESSIVE_APOSTROPHE', 34),
 ('COMMA_PARENTHESIS_WHITESPACE', 39),
 ('SENT_START_CONJUNCTIVE_LINKING_ADVERB_COMMA', 40),
 ('MISSING_HYPHEN', 41),
 ('AM_I', 45),
 ('EN_CONTRACTION_SPELLING', 55),
 ('HE_VERB_AGR', 58),
 ('COMMA_COMPOUND_SENTENCE_2', 90),
 ('EN_COMPOUNDS', 129),
 ('WHITESPACE_RULE', 157),
 ('UPPERCASE_SENTENCE_START', 159),
 ('I_LOWERCASE', 192),
 ('ENGLISH_WORD_REPEAT_BEGINNING_RULE', 426),
 ('COMMA_COMPOUND_SENTENCE', 673),
 ('MORFOLOGIK_RULE_EN_US', 892)]

In [57]:
gam.sort(key = lambda x: x[1])
gam[1011:]

[('POKEMON', 39),
 ('HE_VERB_AGR', 42),
 ('DOUBLE_PUNCTUATION', 44),
 ('SENT_START_CONJUNCTIVE_LINKING_ADVERB_COMMA', 56),
 ('EN_COMPOUNDS', 90),
 ('COMMA_PARENTHESIS_WHITESPACE', 122),
 ('ENGLISH_WORD_REPEAT_BEGINNING_RULE', 124),
 ('COMMA_COMPOUND_SENTENCE_2', 126),
 ('EN_SPECIFIC_CASE', 168),
 ('EN_CONTRACTION_SPELLING', 181),
 ('WHITESPACE_RULE', 234),
 ('UPPERCASE_SENTENCE_START', 245),
 ('I_LOWERCASE', 398),
 ('COMMA_COMPOUND_SENTENCE', 619),
 ('MORFOLOGIK_RULE_EN_US', 2568)]

**Top Errors for each subreddit**:

*Legal Advice* - COMMA_COMPOUND_SENTENCE

*Adulting* - COMMA_COMPOUND_SENTENCE

*Medicine* -  MORFOLOGIK_RULE_EN_US

*Highschool* - MORFOLOGIK_RULE_EN_US

*Broadway* - MORFOLOGIK_RULE_EN_US

*Pittsburgh* - MORFOLOGIK_RULE_EN_US

*Rant* - MORFOLOGIK_RULE_EN_US

*Ccq* -  MORFOLOGIK_RULE_EN_US

*Anime* - MORFOLOGIK_RULE_EN_US

*Eli5* -  MORFOLOGIK_RULE_EN_US

*College* - COMMA_COMPOUND_SENTENCE

*Sports* -  MORFOLOGIK_RULE_EN_US

*Cryptocurrency* - MORFOLOGIK_RULE_EN_US

*Lawyer Talk* - MORFOLOGIK_RULE_EN_US

*Gaming* - MORFOLOGIK_RULE_EN_US

MORFOLOGIK_RULE_EN_US occurs as the top error in 12/15 of the subreddits.

Does this really tell us much, though? Most, if not at all, of the MORFOLOGIK_RULE_EN_US errors are due to spelling errors, and while spelling is important to grammaticality, typos don't really tell me much, especially when 12/15 subreddits top error is MORFOLOGIK_RULE_EN_US. The Anime subreddit had the most MORFOLOGIK_RULE_EN_US errors, with 11050 errors. However, could this be due to names of anime characters that the parser does not recognize? Therefore, while I will consider this in my further analysis, I am also curious as to how this holds for the second most common errors in each subreddit.

 **Second Top Errors for each subreddit**:
 
*Legal Advice* - MORFOLOGIK_RULE_EN_US

*Adulting* - MORFOLOGIK_RULE_EN_US

*Medicine* -  COMMA_COMPOUND_SENTENCE

*Highschool* - I_LOWERCASE

*Broadway* - COMMA_COMPOUND_SENTENCE

*Pittsburgh* - COMMA_COMPOUND_SENTENCE

*Rant* - COMMA_COMPOUND_SENTENCE

*Ccq* -  COMMA_COMPOUND_SENTENCE

*Anime* - COMMA_COMPOUND_SENTENCE

*Eli5* -  COMMA_COMPOUND_SENTENCE

*College* - MORFOLOGIK_RULE_EN_US

*Sports* -  WHITESPACE_RULE

*Cryptocurrency* - WHITESPACE_RULE

*Lawyer Talk* - COMMA_COMPOUND_SENTENCE

*Gaming* - COMMA_COMPOUND_SENTENCE

If the subreddit's top error was MORFOLOGIK_RULE_EN_US before, it turned into COMMA_COMPOUND_SENTENCE for the majority of the subreddits. If it was COMMA_COMPOUND_SENTENCE, then it became MORFOLOGIK_RULE_EN_US. COMMA_COMPOUND_SENTENCE relates to a missing comma for compound sentences, which is very interesting. A lot of people do tend to write run-on sentences online, ignoring the rules they would follow in an academic paper, for example. However, it is interesting that the two top ignored grammaticality areas when typing online are typos and forgetting a comma for a compound sentence. The other cases were the whitespace rule, and I lowercase. Whitespace rule relates to how much whitespace is in between words, sentences, etc. I lower case relates to putting a lowercase i instead of an uppercase one.

In fact, if you look at the top three highest occurring errors for any subreddit, it will be some combination of COMMA_COMPOUND_SENTENCE, WHITESPACE_RULE, MORFOLOGIK_RULE_EN_US, and ENGLISH_WORD_REPEAT_BEGINNING_RULE. Other top 15 errors, that may not occur in other subreddits, will be interesting to explore.

For example, it seems like there is a "POKEMON" error in the gaming subreddit, which I assume is due to the last of accent in the word. There are also "WANNA" and "GONNA" errors, which I assume is due to the use of the "wanna" and "gonna", are found in different subreddits but not all. Specifically, some contain both "WANNA" and "GONNA", some contain one or the other, or some contain neither. There is also an "IDK" error that occurs in some, but not all, of subreddits. These errors are meaningful to me, because they do show a difference in writing among different subreddits. However, I want to explore these errors more, get more detail about them, and see if I can find even deeper analysis of this.

## Analysis To Be Continued